In [ ]:
from pyspark.sql import SparkSession

In [2]:
import pyspark.sql.functions as func

In [3]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType , FloatType, LongType

In [4]:
spark= SparkSession.builder\
    .appName('spotify_de') \
    .getOrCreate()

In [5]:
json_schema = StructType([
    StructField("artists", StringType(), False),
    StructField("duration_ms", LongType(), False),
    StructField("popularity", IntegerType(), False),
    StructField("song_name", StringType(), False),
    StructField("song_preview", StringType(), False)
])

In [6]:
json_df= spark.read \
    .schema(json_schema)\
  .json(r"C:\Users\tapis\PycharmProjects\Spotify_project\API_DATA_CLEAN\final.json")\
  

In [7]:
json_df= json_df.withColumn("duration", func.round(json_df.duration_ms/60000,1))

In [8]:
json_df.createOrReplaceTempView("Songs_table")

In [9]:
selected_df = spark.sql("SELECT *, dense_rank() over(order by popularity desc) as rank_on_chart FROM Songs_table")

In [12]:
selected_df.write.mode("overwrite").parquet("C:/Users/tapis/PycharmProjects/Spotify_project/API_DATA_CLEAN/final.parquet")

In [17]:
selected_df.printSchema()

root
 |-- artists: string (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- song_name: string (nullable = true)
 |-- song_preview: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- rank_on_chart: integer (nullable = false)


In [16]:
json_df.schema.json()

'{"fields":[{"metadata":{},"name":"artists","nullable":true,"type":"string"},{"metadata":{},"name":"duration_ms","nullable":true,"type":"long"},{"metadata":{},"name":"popularity","nullable":true,"type":"integer"},{"metadata":{},"name":"song_name","nullable":true,"type":"string"},{"metadata":{},"name":"song_preview","nullable":true,"type":"string"},{"metadata":{},"name":"duration","nullable":true,"type":"double"}],"type":"struct"}'